In [54]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import (
    BootstrapFewShot,
    BootstrapFewShotWithRandomSearch,
    COPRO,
    MIPROv2,
    BootstrapFinetune,
)

In [34]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
dspy.settings.configure(lm=turbo)

In [35]:
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 72266.47it/s]


In [36]:
for example in gsm8k_trainset:
    print(example)

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})
Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>6000

In [37]:
example = gsm8k_devset[0]
example

Example({'question': '20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.', 'gold_reasoning': "The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles distance while migrating, the combined distance they fly together is 110*20 = <<110*20=2200>>2200 miles.", 'answer': '2200'}) (input_keys={'question'})

In [38]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()

        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)


cot = CoT()
cot

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [39]:
print(example.answer)
cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the total distance the birds traveled in the first season, which is 50 miles. Then, we need to calculate the total distance they traveled in the second season, which is 60 miles. Finally, we can add these two distances together to get the combined distance all of the birds have traveled.',
    answer='110 miles'
)

In [40]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the total distance the birds traveled in the first season, which is 50 miles. Then, we need to calculate the total distance they traveled in the second season, which is 60 miles. Finally, we can add these two distances together to get the combined distance all of the birds have travel

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let's think step by step in order to\x1b calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the total distance the birds traveled in the first season, which is 50 miles. Then, we need to calculate the total distance they traveled in the second season, which is 60 miles. Finally, we can add these two distances together to get the combined distance all of 

In [63]:
fewshot_optimizer = BootstrapFewShot(
    metric=gsm8k_metric,
)

fewshot_cot = fewshot_optimizer.compile(student=cot, trainset=gsm8k_trainset)
fewshot_cot

 40%|████      | 4/10 [00:06<00:09,  1.65s/it]

Bootstrapped 4 full traces after 5 examples in round 0.


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [64]:
print(example.answer)
fewshot_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled. We know that there are 20 birds and they fly from lake Jim to lake Disney, which is 50 miles apart. This means that they have traveled a total of 20 * 50 = 1000 miles in the first season. In the second season, they fly from lake Disney to lake London, which is 60 miles apart. Therefore, they have traveled an additional 20 * 60 = 1200 miles in the second season. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200 miles'
)

In [65]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Reasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 70 - (1/10 * 70) = 63 in Maths. We also know that she scored 20 marks fewer in Maths than she scored in Arts, which means she scored 63 + 20 = 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than she got in Music, which means she scored 70 + 10 = 80 in Social Studies. Therefore, the total number of marks she 

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nReasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 70 - (1/10 * 70) = 63 in Maths. We also know that she scored 20 marks fewer in Maths than she scored in Arts, which means she scored 63 + 20 = 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than she got in Music, which means she scored 70 + 10 = 80 in Social Studies. Therefore, the total num

In [66]:
fewshot_randomsearch_optimizer = BootstrapFewShotWithRandomSearch(
    metric=gsm8k_metric,
)

fewshot_randomsearch_cot = fewshot_randomsearch_optimizer.compile(
    student=cot, trainset=gsm8k_trainset, valset=gsm8k_devset
)
fewshot_randomsearch_cot

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.


Average Metric: 6 / 10  (60.0): 100%|██████████| 10/10 [00:00<00:00, 2667.96it/s]


Score: 60.0 for set: [0]
New best sscore: 60.0 for seed -3
Scores so far: [60.0]
Best score: 60.0


Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:03<00:00,  2.50it/s]


Score: 90.0 for set: [10]
New best sscore: 90.0 for seed -2
Scores so far: [60.0, 90.0]
Best score: 90.0


...

 30%|███       | 3/10 [00:04<00:10,  1.52s/it]


Bootstrapped 2 full traces after 4 examples in round 0.


Average Metric: 7 / 10  (70.0): 100%|██████████| 10/10 [00:03<00:00,  2.81it/s]

Score: 70.0 for set: [10]
Scores so far: [60.0, 90.0, 80.0, 60.0, 60.0, 70.0, 90.0, 70.0, 90.0, 60.0, 50.0, 80.0, 90.0, 60.0, 70.0, 80.0, 50.0, 60.0, 70.0]
Best score: 90.0
Average of max per entry across top 1 scores: 0.9
Average of max per entry across top 2 scores: 0.9
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [67]:
print(example.answer)
fewshot_randomsearch_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Since there are 20 birds, and the distance between the two lakes is 50 miles, the total distance traveled by the birds in the first season is 20 x 50 = 1000 miles. Next, we need to calculate the distance traveled by the birds in the second season, which is from lake Disney to lake London. Since there are still 20 birds, and the distance between the two lakes is 60 miles, the total distance traveled by the birds in the second season is 20 x 60 = 1200 miles. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200'
)

In [68]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.
Answer: 600000

---

Question: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?
Answer: 2

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Answer: 296

---

Question: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.\nAnswer: 600000\n\n---\n\nQuestion: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?\nAnswer: 2\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nAnswer: 296\n\n---\n\nQuestion: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys t

In [56]:
mipro_optimizer = MIPROv2(
    prompt_model=turbo,
    task_model=turbo,
    metric=gsm8k_metric,
)

eval_kwargs = dict(num_threads=6, display_progress=True, display_table=0)
mipro_cot = mipro_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs=eval_kwargs,
)
mipro_cot


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 10 examples in train set * 3 full evals = 780 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

 60%|██████    | 6/10 [00:10<00:07,  1.80s/it]


Bootstrapped 5 full traces after 7 examples in round 0.


 40%|████      | 4/10 [00:05<00:07,  1.27s/it]


Bootstrapped 4 full traces after 5 examples in round 0.


...

Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:03<00:00,  2.90it/s]


FULL TRACE


Average Metric: 1 / 1  (100.0): 100%|██████████| 1/1 [00:00<00:00, 1779.51it/s]
[I 2024-08-06 15:17:54,097] Trial 29 finished with value: 90.0 and parameters: {'0_predictor_instruction': 3, '0_predictor_demos': 1}. Best is trial 16 with value: 100.0.





Use the provided information from the input question to calculate step-by-step and articulate how the given information led to the solution in `answer`.
Example, PatchifiableIn(panel + nonhist_assert.cond_ok + dut_row('.... Quote implied' = answer.
Info: Given the question of "How much did Sam earn last month?", the instruction could prompt the language model with: "Let's think step by step in order to calculate the total earnings for Sam for the previous month. We know from the question that Sam's total earnings is being asked for, so we start by summing up the individual amounts he earned each day. Then, we can add any additional bonuses or deductions for the month to appropriately determine Sam's total earnings.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [57]:
print(example.answer)
mipro_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled. We know that the birds flew 50 miles from lake Jim to lake Disney and then 60 miles from lake Disney to lake London. This means that in total, they flew 50 + 60 = 110 miles. Since there are 20 birds, we can multiply the total distance by 20 to get the combined distance all of the birds have traveled, which is 110 * 20 = 2200 miles.',
    answer='2200 miles'
)

In [58]:
turbo.inspect_history(n=1)




From the given question, generate and provide an appropriate numerical or non-numerical answer by analyzing potential schema combinations and utilizing contextual reasoning.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Reasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 63 in Maths. We also know that she scored 20 marks fewer in Maths than in Arts, so she scored 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than in Music, so she scored 80 in Social Studies. T

"\n\n\nFrom the given question, generate and provide an appropriate numerical or non-numerical answer by analyzing potential schema combinations and utilizing contextual reasoning.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nReasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 63 in Maths. We also know that she scored 20 marks fewer in Maths than in Arts, so she scored 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than in Music, so she scored 80 in 

In [60]:
copro_optimizer = COPRO(
    metric=gsm8k_metric,
)

copro_cot = copro_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs={
        "num_threads": 6,
        "display_progress": True,
    },
)
copro_cot

Average Metric: 1 / 10  (10.0): 100%|██████████| 10/10 [00:10<00:00,  1.07s/it]


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [61]:
print(example.answer)
copro_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance traveled by the birds. First, we need to find the total distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Then, we need to find the total distance traveled in the second season, from lake Disney to lake London. Finally, we can add these two distances together to find the combined distance traveled by all of the birds.',
    answer='The combined distance traveled by the birds in the two seasons is 110 miles.'
)

In [62]:
turbo.inspect_history(n=1)




Experiment with contexts by changing up your starting task and setups that prompts do.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Thoughts: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to calculate the combined distance traveled by the birds. First, we need to find the total distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Then, we need to find the total distance traveled in the second season, from lake Disney to lake London. Finally, we can add these two distances together to find the combined distance t

"\n\n\nExperiment with contexts by changing up your starting task and setups that prompts do.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nThoughts: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let's think step by step in order to\x1b calculate the combined distance traveled by the birds. First, we need to find the total distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Then, we need to find the total distance traveled in the second season, from lake Disney to lake London. Finally, we can add these two distances together to find th